In [23]:
import spotipy
import numpy as np
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import json
import re

In [2]:
cli_cred_manager = SpotifyClientCredentials(client_id="3be9acd2dbcb4943b4573e5659d7f6ba",client_secret="53e274002b594989858aa743b4e81fc3")
sp = spotipy.Spotify(client_credentials_manager=cli_cred_manager)

In [18]:
train_artists = pd.read_csv("../data/train_data_artists.csv")
train_artists

,artist_id,artist_name,artist_popularity,artist_genres
0,1HY2Jd0NmPuamShAr6KMms,Lady Gaga,92,"art pop, dance pop, pop"
1,6qqNVTkY8uBg9cP3Jd7DAH,Billie Eilish,96,"art pop, pop"
2,7GlBOeep6PqTfFi59PTUUN,Chappell Roan,89,indie pop
3,74KM79TiuVKeVCqs8QtB0B,Sabrina Carpenter,94,pop
4,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,96,"canadian contemporary r&b, canadian pop, pop"
...,...,...,...,...
2148,5ZtuBRCHqXY3bSD2BRSYNC,Alexandr Misko,33,acoustic guitar cover
2149,3cGb8b9Zwvka0MWD5RyMDI,Richard Watts,48,acoustic cover
2150,7qVz2CTVWQynqvLaaxZXw6,Matt Skiba,33,"acoustic punk, chicago punk"
2151,5kO4xdEKLuHHHPreu3UmkZ,Blind Willie Johnson,36,"acoustic blues, blues, country blues, gospel b..."


In [21]:
with open("../result/map.json", "r", encoding="utf-8") as f:
    genre_map = json.load(f)

genre_map

{'pop': 'pop',
 'art-pop': 'pop',
 'viral-pop': 'pop',
 'dance-pop': 'pop',
 'synthpop': 'pop',
 'electropop': 'pop',
 'italian-pop': 'pop',
 'classic-uk-pop': 'pop',
 'k-pop': 'pop',
 'french-pop': 'pop',
 'canadian-pop': 'pop',
 'jazz-pop': 'pop',
 'bedroom-pop': 'pop',
 'baroque-pop': 'pop',
 'neo-mellow': 'pop',
 'folk-pop': 'pop',
 'new-romantic': 'pop',
 'indie-pop': 'pop',
 'uk-pop': 'pop',
 'chill-pop': 'pop',
 'pop-rock': 'pop',
 'italian-adult-pop': 'pop',
 'colombian-pop': 'pop',
 'latin-pop': 'pop',
 'puerto-rican-pop': 'pop',
 'singer-songwriter-pop': 'pop',
 'nz-pop': 'pop',
 'korean-pop': 'pop',
 'candy-pop': 'pop',
 'trap-pop': 'pop',
 'k-pop-boy-group': 'pop',
 'k-pop-girl-group': 'pop',
 'vocal-jazz-pop': 'pop',
 'australian-pop': 'pop',
 'modern-country-pop': 'pop',
 'pop-dance': 'pop',
 'classic-italian-pop': 'pop',
 'contemporary-r&b-pop': 'pop',
 'v-pop': 'pop',
 '5th-gen-k-pop': 'pop',
 'korean-ost': 'pop',
 'viet-idol-pop': 'pop',
 'cantopop': 'pop',
 'uk-altern

In [25]:
def generalize_genres(genre_list, mapping):
    generalized_genres = []
    
    for genre_str in genre_list:
        # Split by commas to handle multiple genres per entry
        if not genre_str or genre_str.lower() == 'nan':
            generalized_genres.append("other")
            continue

        subgenres = genre_str.split(',')
        generalized = set()  # Use a set to avoid duplicates
        
        for subgenre in subgenres:
            # Find the general genre if available
            for keyword, general in mapping.items():
                # Use regex to find whole word matches
                if re.search(rf"\b{keyword}\b", subgenre):
                    generalized.add(general)
                    break  # Stop after finding the first match
        
        # Join generalized genres into a single string and add to the result list
        generalized_genres.append(','.join(generalized) if generalized else "other")
    
    return generalized_genres

res = generalize_genres(train_artists["artist_genres"].astype(str), genre_map)

In [27]:
train_artists["generalized_genres"] = res

In [32]:
train_artists[[["other", "pop", "rock", "classical", "blues", "jazz", "hip-hop", "reggae", "electronic", "acoustic", ]]] = 0

In [35]:
for index, artist in train_artists.iterrows():
    splitted_genre = artist["generalized_genres"].split(",")
    for g in splitted_genre:
        train_artists.at[index, g] = 1

In [38]:
genre = []
for i in range(train_artists.shape[0]):
    for j in range(train_artists.shape[1]-1, -1, -1):
        if (train_artists.iloc[i, j] == 1):
            genre.append(train_artists.columns[j])
            break

train_artists["temp_genre"] = genre

In [42]:
train_artists.drop(columns=["generalized_genres"], inplace=True)

In [44]:
train_artists.groupby("temp_genre").count()

,artist_id,artist_name,artist_popularity,artist_genres,other,pop,rock,classical,blues,jazz,hip-hop,reggae,electronic,acoustic
temp_genre,,,,,,,,,,,,,,
acoustic,182,182,182,182,182,182,182,182,182,182,182,182,182,182
blues,170,170,170,170,170,170,170,170,170,170,170,170,170,170
classical,159,159,159,159,159,159,159,159,159,159,159,159,159,159
electronic,151,151,151,151,151,151,151,151,151,151,151,151,151,151
hip-hop,200,200,200,200,200,200,200,200,200,200,200,200,200,200
jazz,316,316,316,316,316,316,316,316,316,316,316,316,316,316
other,95,95,95,95,95,95,95,95,95,95,95,95,95,95
pop,298,298,298,298,298,298,298,298,298,298,298,298,298,298
reggae,179,179,179,179,179,179,179,179,179,179,179,179,179,179


In [ ]:
# train_artists.to_csv("../data/train_data_artists.csv", index=False)